In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
train_path='./train'
test_path='./test'
val_path='./valid'

In [3]:
train_images=[]
train_labels=[]


for i in os.listdir(train_path):
    path=os.path.join(train_path,i)
    if i=='adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib':
        label=0
    elif i=='large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa':
        label=1
    elif i=='normal':
        label=2
    else:
        label=3
        
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j))
        img=cv2.resize(img,(224,224))
        img=np.array(img)
        train_images.append(img)
        train_labels.append(label)        

In [4]:
train_array_images=np.array(train_images)
train_array_labels=np.array(train_labels)

In [5]:
print(train_array_images.shape)
print(train_array_labels.shape)

(613, 224, 224, 3)
(613,)


In [6]:
test_images=[]
test_labels=[]


for i in os.listdir(test_path):
    path=os.path.join(test_path,i)
    if i=='adenocarcinoma':
        label=0
    elif i=='large.cell.carcinoma':
        label=1
    elif i=='normal':
        label=2
    else:
        label=3
        
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j))
        img=cv2.resize(img,(224,224))
        img=np.array(img)
        test_images.append(img)
        test_labels.append(label)  

In [7]:
test_array_images=np.array(test_images)
test_array_labels=np.array(test_labels)

In [8]:
print(test_array_images.shape)
print(test_array_labels.shape)

(315, 224, 224, 3)
(315,)


In [9]:
valid_images=[]
valid_labels=[]


for i in os.listdir(val_path):
    path=os.path.join(val_path,i)
    if i=='adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib':
        label=0
    elif i=='large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa':
        label=1
    elif i=='normal':
        label=2
    else:
        label=3
        
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j))
        img=cv2.resize(img,(224,224))
        img=np.array(img)
        valid_images.append(img)
        valid_labels.append(label) 

In [10]:
valid_array_images=np.array(valid_images)
valid_array_labels=np.array(valid_labels)

In [11]:
print(valid_array_images.shape)
print(valid_array_labels.shape)

(72, 224, 224, 3)
(72,)


In [12]:
model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [13]:
model.trainable=False

In [14]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
flat=Flatten()
dense1=Dense(512,activation='relu')
dense2=Dense(512,activation='relu')
dense3=Dense(512,activation='relu')
output=Dense(4,activation='softmax')

In [16]:
from keras.models import Sequential

model2=Sequential([model,flat,dense1,dense2,dense3,output])

In [17]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 28087620 (107.15 MB)
Trainable params: 133

In [18]:
model2.compile(optimizer='adam' ,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [19]:
# pred_images=model2.predict(test_images)

In [20]:
model2.fit(train_array_images,train_array_labels,epochs=5,validation_data=[valid_array_images,valid_array_labels])

Epoch 1/5


20/20 [==============================] - 144s 7s/step - loss: 19.6822 - accuracy: 0.4290 - val_loss: 8.9165 - val_accuracy: 0.4583
Epoch 2/5
20/20 [==============================] - 139s 7s/step - loss: 1.2128 - accuracy: 0.8630 - val_loss: 3.6791 - val_accuracy: 0.6111
Epoch 3/5
20/20 [==============================] - 139s 7s/step - loss: 0.3894 - accuracy: 0.9396 - val_loss: 2.1811 - val_accuracy: 0.7222
Epoch 4/5
20/20 [==============================] - 139s 7s/step - loss: 0.1093 - accuracy: 0.9821 - val_loss: 1.2921 - val_accuracy: 0.8194
Epoch 5/5
20/20 [==============================] - 139s 7s/step - loss: 0.1258 - accuracy: 0.9837 - val_loss: 1.1816 - val_accuracy: 0.8611


In [36]:
test_image=cv2.imread('./test/normal/normal.png')
# print(test_image)
test_image=cv2.resize(test_image,(224,224))
test_array_image=np.array(test_image)
test_array_image.shape
test_array_image=test_array_image.reshape(1,224,224,3)
test_array_image.shape

(1, 224, 224, 3)

In [37]:
 pred_labels=model2.predict(test_array_image)

1/1 [==============================] - 0s 324ms/step


In [38]:
pred_labels

array([[2.2101402e-24, 5.2234149e-25, 1.0000000e+00, 1.1585815e-18]],
      dtype=float32)

In [39]:
pred_labels.argmax()

2

In [40]:
from joblib import dump

In [41]:
dump(model2,'./../../model/deploy/SavedModels/chest_ct.joblib')

['./../../model/deploy/SavedModels/chest_ct.joblib']